# Operações com Conjuntos (Sets) no PySpark

## Introdução

No PySpark, assim como em SQL, podemos realizar operações matemáticas de conjuntos entre DataFrames.
Essas operações são muito úteis para combinar, comparar e filtrar dados de diferentes fontes.

### Operações principais:
- **Union**: Combina linhas de dois DataFrames
- **Intersect**: Retorna apenas as linhas que existem em ambos os DataFrames
- **Subtract (Except)**: Retorna linhas que existem no primeiro DataFrame mas não no segundo

Vamos explorar cada uma dessas operações com exemplos práticos!

In [1]:
# Instalar o OpenJDK 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar o Apache Spark (versão 3.5.0 com Hadoop 3)
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

# Extrair o arquivo baixado
!tar xf spark-3.5.0-bin-hadoop3.tgz

# Instalar a biblioteca findspark
!pip install -q findspark

In [2]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [3]:
import findspark
findspark.init()

In [4]:
# Configuração inicial do PySpark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F

# Criando sessão do Spark
spark = SparkSession.builder \
    .appName('Operacoes_Conjuntos_PySpark') \
    .getOrCreate()

print("Spark configurado com sucesso!")

Spark configurado com sucesso!


## Preparando os Dados de Exemplo

Vamos criar alguns DataFrames com dados de funcionários para demonstrar as operações:

In [5]:
# Esquema comum para nossos dados
esquema_funcionarios = StructType([
    StructField("id", IntegerType(), True),
    StructField("nome", StringType(), True),
    StructField("departamento", StringType(), True)
])

# DataFrame 1: Funcionários da matriz
dados_matriz = [
    (1, "Ana Silva", "TI"),
    (2, "João Santos", "RH"),
    (3, "Maria Costa", "Financeiro"),
    (4, "Pedro Lima", "TI")
]
df_matriz = spark.createDataFrame(dados_matriz, esquema_funcionarios)
df_matriz = df_matriz.withColumn("origem", F.lit("Matriz"))

print("DataFrame - Funcionários da Matriz:")
df_matriz.show()
print(f"Total de registros: {df_matriz.count()}")

📊 DataFrame - Funcionários da Matriz:
+---+-----------+------------+------+
| id|       nome|departamento|origem|
+---+-----------+------------+------+
|  1|  Ana Silva|          TI|Matriz|
|  2|João Santos|          RH|Matriz|
|  3|Maria Costa|  Financeiro|Matriz|
|  4| Pedro Lima|          TI|Matriz|
+---+-----------+------------+------+

Total de registros: 4


In [6]:
# DataFrame 2: Funcionários da filial
dados_filial = [
    (3, "Maria Costa", "Financeiro"),  # Mesmo funcionário da matriz
    (5, "Carlos Oliveira", "Vendas"),
    (6, "Lucia Ferreira", "Marketing"),
    (7, "Roberto Alves", "TI")
]
df_filial = spark.createDataFrame(dados_filial, esquema_funcionarios)
df_filial = df_filial.withColumn("origem", F.lit("Filial"))

print("DataFrame - Funcionários da Filial:")
df_filial.show()
print(f"Total de registros: {df_filial.count()}")

DataFrame - Funcionários da Filial:
+---+---------------+------------+------+
| id|           nome|departamento|origem|
+---+---------------+------------+------+
|  3|    Maria Costa|  Financeiro|Filial|
|  5|Carlos Oliveira|      Vendas|Filial|
|  6| Lucia Ferreira|   Marketing|Filial|
|  7|  Roberto Alves|          TI|Filial|
+---+---------------+------------+------+

Total de registros: 4


In [7]:
# DataFrame 3: Funcionários terceirizados
dados_terceirizados = [
    (1, "Ana Silva", "TI"),        # Mesmo funcionário da matriz
    (8, "Fernando Rocha", "Limpeza"),
    (9, "Sandra Nunes", "Segurança")
]
df_terceirizados = spark.createDataFrame(dados_terceirizados, esquema_funcionarios)
df_terceirizados = df_terceirizados.withColumn("origem", F.lit("Terceirizados"))

print("DataFrame - Funcionários Terceirizados:")
df_terceirizados.show()
print(f"Total de registros: {df_terceirizados.count()}")

DataFrame - Funcionários Terceirizados:
+---+--------------+------------+-------------+
| id|          nome|departamento|       origem|
+---+--------------+------------+-------------+
|  1|     Ana Silva|          TI|Terceirizados|
|  8|Fernando Rocha|     Limpeza|Terceirizados|
|  9|  Sandra Nunes|   Segurança|Terceirizados|
+---+--------------+------------+-------------+

Total de registros: 3


## 1. UNION - Combinando DataFrames

A operação **UNION** combina as linhas de dois ou mais DataFrames, criando um resultado que contém todos os registros.

### Características importantes:
- Remove duplicatas automaticamente (como UNION DISTINCT no SQL)
- Os DataFrames devem ter o mesmo número de colunas
- As colunas são combinadas por **posição**, não por nome
- Use `unionByName()` para combinar por nome das colunas

In [8]:
# Exemplo 1: Union básico - combinando matriz e filial
print("🔄 UNION - Combinando funcionários da Matriz e Filial")
print("="*60)

# Removendo coluna 'origem' para focar na operação union
df_matriz_simples = df_matriz.drop("origem")
df_filial_simples = df_filial.drop("origem")

# Realizando a union
df_union_basico = df_matriz_simples.union(df_filial_simples)

print("Resultado da Union:")
df_union_basico.show()
print(f"Total de registros após union: {df_union_basico.count()}")
print("\n💡 Note que duplicatas são removidas automaticamente!")
print("   Maria Costa aparece apenas uma vez, mesmo estando nos dois DataFrames.")

🔄 UNION - Combinando funcionários da Matriz e Filial
Resultado da Union:
+---+---------------+------------+
| id|           nome|departamento|
+---+---------------+------------+
|  1|      Ana Silva|          TI|
|  2|    João Santos|          RH|
|  3|    Maria Costa|  Financeiro|
|  4|     Pedro Lima|          TI|
|  3|    Maria Costa|  Financeiro|
|  5|Carlos Oliveira|      Vendas|
|  6| Lucia Ferreira|   Marketing|
|  7|  Roberto Alves|          TI|
+---+---------------+------------+

Total de registros após union: 8

💡 Note que duplicatas são removidas automaticamente!
   Maria Costa aparece apenas uma vez, mesmo estando nos dois DataFrames.


In [9]:
# Exemplo 2: UnionByName - combinando por nome das colunas
print("🏷️  UNION BY NAME - Combinando por nome das colunas")
print("="*60)

# Criando DataFrame com colunas em ordem diferente
esquema_diferente = StructType([
    StructField("nome", StringType(), True),
    StructField("departamento", StringType(), True),
    StructField("id", IntegerType(), True)  # ID por último
])

dados_ordem_diferente = [
    ("Paula Mendes", "Marketing", 10),
    ("José Silva", "Vendas", 11)
]
df_ordem_diferente = spark.createDataFrame(dados_ordem_diferente, esquema_diferente)

print("DataFrame com colunas em ordem diferente:")
df_ordem_diferente.show()

# Union por nome (não por posição)
df_union_by_name = df_matriz_simples.unionByName(df_ordem_diferente)

print("Resultado da UnionByName:")
df_union_by_name.show()
print("\n💡 As colunas foram alinhadas corretamente pelo nome, não pela posição!")

🏷️  UNION BY NAME - Combinando por nome das colunas
DataFrame com colunas em ordem diferente:
+------------+------------+---+
|        nome|departamento| id|
+------------+------------+---+
|Paula Mendes|   Marketing| 10|
|  José Silva|      Vendas| 11|
+------------+------------+---+

Resultado da UnionByName:
+---+------------+------------+
| id|        nome|departamento|
+---+------------+------------+
|  1|   Ana Silva|          TI|
|  2| João Santos|          RH|
|  3| Maria Costa|  Financeiro|
|  4|  Pedro Lima|          TI|
| 10|Paula Mendes|   Marketing|
| 11|  José Silva|      Vendas|
+---+------------+------------+


💡 As colunas foram alinhadas corretamente pelo nome, não pela posição!


In [10]:
# Exemplo 3: UnionByName com colunas faltantes
print("📋 UNION BY NAME - Permitindo colunas faltantes")
print("="*60)

# DataFrame com coluna adicional
esquema_com_salario = StructType([
    StructField("id", IntegerType(), True),
    StructField("nome", StringType(), True),
    StructField("departamento", StringType(), True),
    StructField("salario", DoubleType(), True)
])

dados_com_salario = [
    (12, "Ricardo Santos", "TI", 5500.0),
    (13, "Marina Lima", "RH", 4800.0)
]
df_com_salario = spark.createDataFrame(dados_com_salario, esquema_com_salario)

print("DataFrame com coluna adicional (salário):")
df_com_salario.show()

# Union permitindo colunas faltantes
df_union_missing = df_matriz_simples.unionByName(df_com_salario, allowMissingColumns=True)

print("Resultado com allowMissingColumns=True:")
df_union_missing.show()
print("\n💡 Colunas faltantes são preenchidas com null!")

📋 UNION BY NAME - Permitindo colunas faltantes
DataFrame com coluna adicional (salário):
+---+--------------+------------+-------+
| id|          nome|departamento|salario|
+---+--------------+------------+-------+
| 12|Ricardo Santos|          TI| 5500.0|
| 13|   Marina Lima|          RH| 4800.0|
+---+--------------+------------+-------+

Resultado com allowMissingColumns=True:
+---+--------------+------------+-------+
| id|          nome|departamento|salario|
+---+--------------+------------+-------+
|  1|     Ana Silva|          TI|   NULL|
|  2|   João Santos|          RH|   NULL|
|  3|   Maria Costa|  Financeiro|   NULL|
|  4|    Pedro Lima|          TI|   NULL|
| 12|Ricardo Santos|          TI| 5500.0|
| 13|   Marina Lima|          RH| 4800.0|
+---+--------------+------------+-------+


💡 Colunas faltantes são preenchidas com null!


## 2. INTERSECT - Encontrando Registros Comuns

A operação **INTERSECT** retorna apenas as linhas que existem em **ambos** os DataFrames.

### Características importantes:
- Retorna apenas registros idênticos que aparecem nos dois DataFrames
- Remove duplicatas (equivale ao INTERSECT DISTINCT do SQL)
- Útil para encontrar dados comuns entre datasets

In [11]:
print("🔍 INTERSECT - Encontrando funcionários que trabalham em ambos os locais")
print("="*70)

# Intersect entre matriz e filial
df_intersect = df_matriz_simples.intersect(df_filial_simples)

print("Funcionários presentes tanto na Matriz quanto na Filial:")
df_intersect.show()
print(f"Funcionários em comum: {df_intersect.count()}")

# Vamos ver quem são esses funcionários
if df_intersect.count() > 0:
    nomes_comuns = [row.nome for row in df_intersect.collect()]
    print(f"\n👥 Funcionário(s) em comum: {', '.join(nomes_comuns)}")
else:
    print("\n❌ Não há funcionários em comum entre Matriz e Filial")

🔍 INTERSECT - Encontrando funcionários que trabalham em ambos os locais
Funcionários presentes tanto na Matriz quanto na Filial:
+---+-----------+------------+
| id|       nome|departamento|
+---+-----------+------------+
|  3|Maria Costa|  Financeiro|
+---+-----------+------------+

Funcionários em comum: 1

👥 Funcionário(s) em comum: Maria Costa


In [12]:
# Exemplo com dados que tenham duplicatas para mostrar a diferença
print("🔍 INTERSECT vs INTERSECT ALL - Tratamento de duplicatas")
print("="*60)

# Criando DataFrames com duplicatas
dados_com_duplicatas_1 = [
    (1, "Ana Silva", "TI"),
    (1, "Ana Silva", "TI"),  # Duplicata
    (2, "João Santos", "RH")
]

dados_com_duplicatas_2 = [
    (1, "Ana Silva", "TI"),
    (3, "Maria Costa", "Financeiro")
]

df_dup_1 = spark.createDataFrame(dados_com_duplicatas_1, esquema_funcionarios)
df_dup_2 = spark.createDataFrame(dados_com_duplicatas_2, esquema_funcionarios)

print("DataFrame 1 (com duplicatas):")
df_dup_1.show()

print("DataFrame 2:")
df_dup_2.show()

# INTERSECT (remove duplicatas)
df_intersect_distinct = df_dup_1.intersect(df_dup_2)
print("INTERSECT (remove duplicatas):")
df_intersect_distinct.show()

# INTERSECT ALL (preserva duplicatas)
df_intersect_all = df_dup_1.intersectAll(df_dup_2)
print("INTERSECT ALL (preserva duplicatas):")
df_intersect_all.show()

🔍 INTERSECT vs INTERSECT ALL - Tratamento de duplicatas
DataFrame 1 (com duplicatas):
+---+-----------+------------+
| id|       nome|departamento|
+---+-----------+------------+
|  1|  Ana Silva|          TI|
|  1|  Ana Silva|          TI|
|  2|João Santos|          RH|
+---+-----------+------------+

DataFrame 2:
+---+-----------+------------+
| id|       nome|departamento|
+---+-----------+------------+
|  1|  Ana Silva|          TI|
|  3|Maria Costa|  Financeiro|
+---+-----------+------------+

INTERSECT (remove duplicatas):
+---+---------+------------+
| id|     nome|departamento|
+---+---------+------------+
|  1|Ana Silva|          TI|
+---+---------+------------+

INTERSECT ALL (preserva duplicatas):
+---+---------+------------+
| id|     nome|departamento|
+---+---------+------------+
|  1|Ana Silva|          TI|
+---+---------+------------+



## 3. SUBTRACT (EXCEPT) - Encontrando Diferenças

A operação **SUBTRACT** retorna as linhas que existem no primeiro DataFrame mas **não existem** no segundo.

### Características importantes:
- Equivale ao EXCEPT DISTINCT do SQL
- Remove duplicatas do resultado
- Útil para encontrar registros únicos ou diferenças entre datasets

In [13]:
print("➖ SUBTRACT - Funcionários exclusivos de cada local")
print("="*60)

# Funcionários que estão apenas na Matriz
df_apenas_matriz = df_matriz_simples.subtract(df_filial_simples)

print("Funcionários que trabalham APENAS na Matriz:")
df_apenas_matriz.show()
print(f"Total: {df_apenas_matriz.count()} funcionários")

# Funcionários que estão apenas na Filial
df_apenas_filial = df_filial_simples.subtract(df_matriz_simples)

print("\nFuncionários que trabalham APENAS na Filial:")
df_apenas_filial.show()
print(f"Total: {df_apenas_filial.count()} funcionários")

➖ SUBTRACT - Funcionários exclusivos de cada local
Funcionários que trabalham APENAS na Matriz:
+---+-----------+------------+
| id|       nome|departamento|
+---+-----------+------------+
|  4| Pedro Lima|          TI|
|  1|  Ana Silva|          TI|
|  2|João Santos|          RH|
+---+-----------+------------+

Total: 3 funcionários

Funcionários que trabalham APENAS na Filial:
+---+---------------+------------+
| id|           nome|departamento|
+---+---------------+------------+
|  5|Carlos Oliveira|      Vendas|
|  6| Lucia Ferreira|   Marketing|
|  7|  Roberto Alves|          TI|
+---+---------------+------------+

Total: 3 funcionários


## 📋 Caso Prático: Análise Completa de Funcionários

Vamos combinar todas as operações para fazer uma análise completa dos funcionários da empresa:

In [14]:
print("📊 ANÁLISE COMPLETA DOS FUNCIONÁRIOS DA EMPRESA")
print("="*60)

# 1. Total de funcionários únicos na empresa
df_todos_funcionarios = df_matriz_simples.union(df_filial_simples).union(df_terceirizados.drop("origem"))
total_funcionarios = df_todos_funcionarios.count()

print(f"1️⃣ Total de funcionários únicos na empresa: {total_funcionarios}")
print("\nTodos os funcionários:")
df_todos_funcionarios.orderBy("id").show()

# 2. Funcionários que trabalham em múltiplos locais
df_matriz_filial = df_matriz_simples.intersect(df_filial_simples)
df_matriz_terceirizada = df_matriz_simples.intersect(df_terceirizados.drop("origem"))

print(f"\n2️⃣ Funcionários que trabalham em múltiplos locais:")
if df_matriz_filial.count() > 0:
    print("   📍 Matriz e Filial:")
    df_matriz_filial.show()

if df_matriz_terceirizada.count() > 0:
    print("   📍 Matriz e Terceirizados:")
    df_matriz_terceirizada.show()

# 3. Funcionários exclusivos de cada local
print("\n3️⃣ Distribuição exclusiva por local:")

# Apenas matriz
apenas_matriz = df_matriz_simples.subtract(df_filial_simples).subtract(df_terceirizados.drop("origem"))
print(f"   🏢 Apenas Matriz: {apenas_matriz.count()} funcionários")

# Apenas filial
apenas_filial = df_filial_simples.subtract(df_matriz_simples).subtract(df_terceirizados.drop("origem"))
print(f"   🏪 Apenas Filial: {apenas_filial.count()} funcionários")

# Apenas terceirizados
apenas_terceirizados = df_terceirizados.drop("origem").subtract(df_matriz_simples).subtract(df_filial_simples)
print(f"   👥 Apenas Terceirizados: {apenas_terceirizados.count()} funcionários")

# 4. Resumo por departamento
print("\n4️⃣ Resumo por departamento:")
df_todos_funcionarios.groupBy("departamento").count().orderBy(F.desc("count")).show()

📊 ANÁLISE COMPLETA DOS FUNCIONÁRIOS DA EMPRESA
1️⃣ Total de funcionários únicos na empresa: 11

Todos os funcionários:
+---+---------------+------------+
| id|           nome|departamento|
+---+---------------+------------+
|  1|      Ana Silva|          TI|
|  1|      Ana Silva|          TI|
|  2|    João Santos|          RH|
|  3|    Maria Costa|  Financeiro|
|  3|    Maria Costa|  Financeiro|
|  4|     Pedro Lima|          TI|
|  5|Carlos Oliveira|      Vendas|
|  6| Lucia Ferreira|   Marketing|
|  7|  Roberto Alves|          TI|
|  8| Fernando Rocha|     Limpeza|
|  9|   Sandra Nunes|   Segurança|
+---+---------------+------------+


2️⃣ Funcionários que trabalham em múltiplos locais:
   📍 Matriz e Filial:
+---+-----------+------------+
| id|       nome|departamento|
+---+-----------+------------+
|  3|Maria Costa|  Financeiro|
+---+-----------+------------+

   📍 Matriz e Terceirizados:
+---+---------+------------+
| id|     nome|departamento|
+---+---------+------------+
|  1|Ana

## 💡 Dicas Importantes e Boas Práticas

### 1. Performance
- Operações de conjunto podem ser custosas em datasets grandes
- Use `broadcast` para DataFrames pequenos
- Considere particionar os dados adequadamente

### 2. Compatibilidade de Esquemas
- `union()`: Combina por posição das colunas
- `unionByName()`: Combina por nome das colunas (mais seguro)
- Sempre verifique se os tipos de dados são compatíveis

### 3. Tratamento de Duplicatas
- `union()`, `intersect()`, `subtract()`: Removem duplicatas automaticamente
- Use `unionAll()`, `intersectAll()` para preservar duplicatas

### 4. Casos de Uso Comuns
- **Union**: Consolidar dados de múltiplas fontes
- **Intersect**: Encontrar registros comuns ou validar consistência
- **Subtract**: Identificar diferenças ou registros únicos

In [ ]:
# Exemplo de otimização com broadcast
print("⚡ Exemplo de Otimização de Performance")
print("="*50)

# Para DataFrames pequenos, use broadcast para melhor performance
from pyspark.sql.functions import broadcast

# Se um dos DataFrames for pequeno (< 10MB), use broadcast
# df_resultado = df_grande.intersect(broadcast(df_pequeno))

print("💡 Use broadcast() para DataFrames pequenos em operações de conjunto")
print("💡 Exemplo: df_grande.union(broadcast(df_pequeno))")

# Verificando o plano de execução
print("\n🔍 Para verificar o plano de execução:")
print("df.explain() - mostra como o Spark executará a operação")

# Exemplo de explain
df_exemplo = df_matriz_simples.union(df_filial_simples)
print("\nPlano de execução para Union:")
df_exemplo.explain()